In [ ]:
!python example_flow.py run --n 9
!python upload_folder_flow.py run --dir .

In [ ]:
import metaflow
import pandas as pd
import opal.flow

s3 = opal.flow.minio_s3fs()

# test that local shared metadata is configured correctly
assert metaflow.get_metadata() == "local@/opt/opal/metaflow-metadata"

ex_flow = metaflow.Flow("ExampleFlow")
up_flow = metaflow.Flow("UploadFlow")
ex_run = ex_flow.latest_successful_run
up_run = up_flow.latest_successful_run

# test that files in data_files actually exist
assert s3.ls(up_run.data.data_files['file'])
assert s3.exists(ex_run.data.data_files['parquet_table'])

pd.read_parquet(ex_run.data.data_files['parquet_table'], filesystem=s3)


In [ ]:
from opal.query import Instance
import opal.publish

instance = Instance()
result = instance._type('ExampleFlow').search()

# clean up old catalog data for the following tests
for r in result.all():
    opal.publish.delete(r['kind_id'])

opal.flow.publish_run(ex_run)
opal.flow.publish_run(up_run)
result = instance._type('ExampleFlow').search()

display(result.all()[0])

# test that flows were published to the catalog
assert len(result.all()) > 0
assert all([ ex_flow[r["kind_id"]] for r in result.all() ])


In [ ]:
opal.flow.flow_script_utils.delete_run_data(up_run)

# test the following:
# tagged as 'no_data'
assert 'no_data' in up_run.tags
# data files deleted
assert not s3.ls(up_run.data.data_files['file'])
# removed from catalog
assert not instance.one(up_run.id).all()

up_run.tags